In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import layers

import utils
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

In [ ]:
class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size = 2, strides = 2):
        super(Block, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions

        for i in range(self.repetitions):
            vars(self)[f'conv2d_{i}'] = tf.keras.layers.Conv2D(self.filters, 
            self.kernel_size, activation = tf.nn.relu, padding = 'same')
        
        self.max_pooling = tf.keras.layers.MaxPool2D(pool_size = pool_size, strides = strides)
            
    def call(self, inputs):
        conv2d_0 = vars(self)['conv2d_0']

        x = conv2d_0(inputs)

        for i in range(1, self.repetitions):
            conv2d_i = vars(self)[f'conv2d_{i}']
            x = conv2d_i(x)
        
        max_pooling = self.max_pooling(x)
        return max_pooling

In [ ]:
class VGG(tf.keras.Model):
    def __init__(self, num_classes):
        super(VGG, self).__init__()

        self.block_a = Block(filters = 32, kernel_size = 3, repetitions = 2)
        self.block_b = Block(filters = 64, kernel_size = 3, repetitions = 2)
        self.block_c = Block(filters = 128, kernel_size = 3, repetitions = 2)
        self.block_d = Block(filters = 256, kernel_size = 3, repetitions = 2)
        
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(128, activation = tf.nn.relu)
        self.classifier = tf.keras.layers.Dense(num_classes, activation = 'softmax')

    def call(self, inputs):
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x


In [ ]:
dataset = tfds.load('cats_vs_dogs', split = tfds.Split.TRAIN, data_dir = 'data/')
vgg = VGG(num_classes = 2)
vgg.compile(optimizer =  tf.keras.optimizers.Adam(), loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics = ['accuracy'])

def preprocess(features):
    image = tf.image.resize(features['image'], (224, 224))
    return tf.cast(image, tf.float32) / 255.0, features['label']

dataset = dataset.map(preprocess).batch(32)
vgg.fit(dataset, epochs = 10)